# 2.스파크 기초

## 2.2 첫 스파크 프로그램 작성
### 2.2.2 첫 코드 예제

In [1]:
val licLines = sc.textFile("/usr/local/spark/LICENSE")

Intitializing Scala interpreter ...

Spark Web UI available at http://4ad35e748219:4040
SparkContext available as 'sc' (version = 3.0.0, master = local[*], app id = local-1602344573672)
SparkSession available as 'spark'


licLines: org.apache.spark.rdd.RDD[String] = /usr/local/spark/LICENSE MapPartitionsRDD[1] at textFile at <console>:25


In [2]:
val lineCnt = licLines.count

lineCnt: Long = 582


In [3]:
// BSD문자가 등장한 줄만 포함(익명 함수)

val bsdLines = licLines.filter(line => line.contains("BSD"))
bsdLines.count

bsdLines: org.apache.spark.rdd.RDD[String] = MapPartitionsRDD[2] at filter at <console>:28
res0: Long = 3


In [4]:
// BSD문자가 등장한 줄만 포함(일반 함수)

def isBSD(line: String) = {line.contains("BSD")}
val bsdLines = licLines.filter(isBSD)
bsdLines.count

isBSD: (line: String)Boolean
bsdLines: org.apache.spark.rdd.RDD[String] = MapPartitionsRDD[3] at filter at <console>:31
res1: Long = 3


In [5]:
// 한 줄씩 출력
bsdLines.foreach(bLine => println(bLine))
// 동일한 표현 bsdLines.foreach(println)

BSD 2-Clause
BSD 3-Clause
is distributed under the 3-Clause BSD license.


### 2.2.3 RDD의 개념
- 불변성 : 읽기 전용
    - 바뀌어야 하면 새로 만든다
- 복원성 : 장애 내성
    - RDD 계보(lineage) 를 유지해서 이전의 변환 결과물을 복구할 수 있다.
- 분산 : 노드 한 개 이상에 저장된 데이터 셋

## 2.3 RDD의 기본 행동 연산자 및 변환 연산자
### 2.3.1 map 변환 연산자

- seq 자료형을 제곱하기

In [6]:
// 스칼라 문법인 듯
10 to 50 by 10

res3: scala.collection.immutable.Range = Range 10 to 50 by 10


In [7]:
val number = sc.parallelize(10 to 50 by 10)

number: org.apache.spark.rdd.RDD[Int] = ParallelCollectionRDD[4] at parallelize at <console>:25


In [8]:
number.foreach(x => println(x))

10
30
50
20
40


In [9]:
val numberSquared = number.map(num => num*num)
numberSquared.foreach(println)

900
2500
400
1600
100


numberSquared: org.apache.spark.rdd.RDD[Int] = MapPartitionsRDD[5] at map at <console>:26


- 문자 뒤집기

In [10]:
val reversed = numberSquared.map(x => x.toString.reverse)

reversed: org.apache.spark.rdd.RDD[String] = MapPartitionsRDD[6] at map at <console>:26


In [11]:
reversed.foreach(x => println(x))

0052
004
001
0061
009


In [12]:
// 플레이스 홀더(_)를 이용한 간결한 표현
val alsoReversed = numberSquared.map(_.toString.reverse)
alsoReversed.first

alsoReversed: org.apache.spark.rdd.RDD[String] = MapPartitionsRDD[7] at map at <console>:27
res7: String = 001


In [13]:
alsoReversed.top(4)

res8: Array[String] = Array(009, 0061, 0052, 004)


### 2.3.2 distinct, flatMap

- 중복 ID를 제거하여 고객의 수 파악하기

In [14]:
val lines = sc.textFile("ch2_client_ids.log")
val idsStr = lines.map(line => line.split(","))
idsStr.foreach(println(_))

[Ljava.lang.String;@38d3e491
[Ljava.lang.String;@7d51e44b
[Ljava.lang.String;@750b2ca2
[Ljava.lang.String;@c53efd4


lines: org.apache.spark.rdd.RDD[String] = ch2_client_ids.log MapPartitionsRDD[10] at textFile at <console>:25
idsStr: org.apache.spark.rdd.RDD[Array[String]] = MapPartitionsRDD[11] at map at <console>:26


In [15]:
idsStr.first

res10: Array[String] = Array(15, 16, 20, 20)


In [16]:
idsStr.collect // 배열의 배열이 반환됨

res11: Array[Array[String]] = Array(Array(15, 16, 20, 20), Array(77, 80, 94), Array(94, 98, 16, 31), Array(31, 15, 20))


- flatMap : collect와 차이점 파악하기

In [17]:
val ids = lines.flatMap(_.split(","))
ids.collect // 일반배 열이 반환됨

ids: org.apache.spark.rdd.RDD[String] = MapPartitionsRDD[12] at flatMap at <console>:26
res12: Array[String] = Array(15, 16, 20, 20, 77, 80, 94, 94, 98, 16, 31, 31, 15, 20)


In [18]:
ids.first

res13: String = 15


In [19]:
ids.collect.mkString(";") // array를 string으로 변환할 때 인수의 문자를 사용함

res14: String = 15;16;20;20;77;80;94;94;98;16;31;31;15;20


In [20]:
val intIds = ids.map(_.toInt)
intIds.collect

intIds: org.apache.spark.rdd.RDD[Int] = MapPartitionsRDD[13] at map at <console>:26
res15: Array[Int] = Array(15, 16, 20, 20, 77, 80, 94, 94, 98, 16, 31, 31, 15, 20)


- distinct

In [21]:
val uniqueIds = intIds.distinct
uniqueIds.collect

uniqueIds: org.apache.spark.rdd.RDD[Int] = MapPartitionsRDD[16] at distinct at <console>:26
res16: Array[Int] = Array(16, 80, 98, 20, 94, 15, 77, 31)


In [22]:
val infalCount = uniqueIds.count // 고객 수

infalCount: Long = 8


In [23]:
val transactionCount = intIds.count // 거래 수

transactionCount: Long = 14


### 2.3.3 sample. take, takeSample 연산으로 RDD의 일부요소 가져오기

- 샘플링

In [24]:
val s = uniqueIds.sample(false, 0.3) // 비복원

s: org.apache.spark.rdd.RDD[Int] = PartitionwiseSampledRDD[17] at sample at <console>:26


In [25]:
s.count

res17: Long = 1


In [26]:
s.collect

res18: Array[Int] = Array(31)


In [27]:
val swr = uniqueIds.sample(true, 0.5) // 복원
swr.count

swr: org.apache.spark.rdd.RDD[Int] = PartitionwiseSampledRDD[18] at sample at <console>:26
res19: Long = 4


In [28]:
swr.collect

res20: Array[Int] = Array(16, 80, 15, 15)


In [29]:
val taken = uniqueIds.takeSample(false, 5) // takeSample

taken: Array[Int] = Array(20, 16, 15, 31, 80)


- take : 결과의 일부만 간단하게 확인할 때 유용함

In [30]:
uniqueIds.take(3)

res21: Array[Int] = Array(16, 80, 98)


## 2.4 Dounble RDD 전용 함수
### 2.4.1 기초 통계량 계산

In [31]:
intIds.mean

res22: Double = 44.785714285714285


In [32]:
intIds.sum

res23: Double = 627.0


In [33]:
intIds.variance

res24: Double = 1114.8826530612243


In [34]:
intIds.stdev

res25: Double = 33.3898585361068


### 2.4.2 히스토그램

In [35]:
intIds.histogram(Array(1.0, 50.0, 100.0)) // 각 구간에 속한 요소의 갯수를 반환

res26: Array[Long] = Array(9, 5)


In [36]:
intIds.histogram(3) // 각 구간의 경계 배열과 요소의 갯수를 같이 반환

res27: (Array[Double], Array[Long]) = (Array(15.0, 42.66666666666667, 70.33333333333334, 98.0),Array(9, 0, 5))


### 2.4.3 근사 합계 및 평균 계산

- 밀리초 단위의 제한 시간을 인자로 받아 데서드가 실행될 최대 시간을 결정

In [37]:
intIds.sumApprox(100, 0.95)

res28: org.apache.spark.partial.PartialResult[org.apache.spark.partial.BoundedDouble] = (final: [627.000, 627.000])


In [38]:
intIds.meanApprox(100, 0.95)

res29: org.apache.spark.partial.PartialResult[org.apache.spark.partial.BoundedDouble] = (final: [44.786, 44.786])
